# Clean Data

* Not all the comments in the thread are job postings. Some are questions and comments too. I could see the conversations that happen around the posting containing useful information, but 
* mark posts that aren't job postings

In [ ]:
%%capture
#| default_exp clean

%pip install openai
%pip install numpy
# %pip install langchain

We need to determine:
* company name
* company location
* contact info

If any information isn't present, we need to make sure that we don't hallucinate.

## Is the posting a job posting?

In [ ]:
# | export
import os
import openai

# openai.api_base = "https://openrouter.ai/api/v1"
openai.api_key = os.environ.get('OPENAI_API_KEY')

def getResponse(text: str):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": text}],
    )

    result = {
        "input_text": text,
        "text": response.choices[0].message.content
    }

    return result

In [ ]:
# |export
import json

def toJson(response, repromptNum=None):
    # try catch here. If item fails, rerun it. If it fails more than 3 times, return None
    try:
        return json.loads(response.replace('\n',''))
    except:
        if repromptNum == None:
            repromptNum = 0
        
        if repromptNum > 3:
            return '{ "errpr": "Could not receive valid json: Too many reprompts","item": '+item+'}'
        
        repromptNum += 1
        return reprompt(response, repromptNum)
    
def reprompt(item, repromptNum=None):
    newPrompt = 'Your response: \n\n'+item.text+'was not valid json. Please try again. Here is the original request:\n' + item

    response = getResponse(newPrompt)

    print('response')
    return toJson(response['text'], repromptNum=repromptNum)


In [ ]:
#| export

def isJob(item, resultItem):
    text='Is the following message a job posting, or something else? If it is a job posting, reply { "isPosting: true }. If it is not a job posting, reply with { "isPosting": false }\n-----------------\n'+ item['comment']

    result = getResponse(text)
    resp = toJson(result['text'])
 
    resultItem['isPosting'] = resp['isPosting']

    return resultItem

In [ ]:
#| export  

# Determine if the items are jobs
def processItems(items):

    results = []

    for item in items:
        results.append(isJob(item))

    return results

In [1]:
# | hide
from nbdev import nbdev_export

nbdev_export()
